# Background

Interactive Brokers don't show the **Return on Margin (RoM)** and **Probability of the Return (PoR)** in its daily reports.

This program extracts information from the daily activity report and puts the RoM and PoR for each strategy.

# Preparation

1. Log on to [ibkr.com](https://www.interactivebrokers.com.hk/en/home.php). Click on *Reports -> Activity -> Statements*
       Steps 2 to 9 to be done only once
2. Create a new **Customize Statements**. Name it as __*OptionsOnly*__.
3. Select tick boxes agains *Open Positions, Dividends* and *Option Exercises/Assignments* only. Leave all others blank
4. In **Profit and Loss:** - Choose *MTM and Realized P/L*
5. In **Statement Type:** - Choose *Activity Statement*
6. Choose your **Account:**
7. Choose **Activity Period:** as *Daily*
8. Choose **Format:** to be *CSV*
9. Save

## Run the report

10\. Choose *OptionsOnly* from Statement, **Format** as *CSV*  
11\. Run the report.  
12\. Rename the file to __*Yesterday.csv*__



# The Program

In [46]:
''' Program to extract stocks and options from IBKR Activity Statement
    Generated from ibkr.com -> Reports -> Activity Statements -> "Options Only"
    Choose previous day's report in CSV format

    Date: 18 June 2017     Rev: 1.0
'''

import csv
import pandas as pd

raw_data = []                      # raw_data list extracted from yesterday's csv file
clean_data = []                    # data cleansed from raw_data

with open('Yesterday.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',' )

    for row in reader:

        # filter for "Open Positions" without "Total"

        if (str(row[0]) == "Open Positions" and str(row[1]) != "Total" ):
            raw_data.append(row)    # Append raw_data list

# Rename dataframe header with first element of raw_data
s = pd.DataFrame(raw_data)
s.columns = s.iloc[0]
s = s[1:]

# Remove excess DataDiscriminator rows
s = s[s.DataDiscriminator != "DataDiscriminator" ]

# Replace "Open" column with date from the next row for "Summary"

s.loc[ s.DataDiscriminator=='Summary','Open'] = s.Open.shift(-1).str.slice(0,10)
s = s[ s.DataDiscriminator=='Summary' ]


s = s[s.DataDiscriminator == "Summary"] # Keep only the Summary rows in the dataset

s['Asset Category'] = s['Asset Category'].str.replace('Equity and Index Options','Options') # Shorten "Options" in Category

# Simplify dataframe field names
s.columns.values[3] = 'Category'
s.columns.values[9] = 'Cost'
s.columns.values[11] = 'Close'

# Assign and Split Options into separate columns
s = s.assign(**s.Symbol.str.split(' ', expand=True).rename(columns={0:'Symbol', 1:'Expiry', 2:'Strike', 3:'Right'}))

# Assemble the most important fields
clean_data = s[['Open', 'Category', 'Currency','Symbol', 'Expiry', 'Right', 'Strike', 'Quantity', 'Mult', 'Cost', 'Close']]

clean_data

,Open,Category,Currency,Symbol,Expiry,Right,Strike,Quantity,Mult,Cost,Close
1,2017-05-12,Stocks,USD,AKAM,None,None,None,200,1,49.22696796,51.3
3,2017-03-31,Stocks,USD,APA,None,None,None,1900,1,50.607305856,46.08
8,2017-06-02,Options,USD,AA,30JUN17,P,27.0,17,100,0.127931,0.0274
11,2017-06-02,Options,USD,AA,30JUN17,P,28.0,-17,100,0.172045076,0.0645
14,2017-06-13,Options,USD,AABA,21JUL17,C,60.0,-7,100,0.34204137,0.1576
17,2017-06-13,Options,USD,AABA,21JUL17,C,62.5,7,100,0.217931,0.0749
20,2017-05-24,Options,USD,AAPL,30JUN17,P,142.0,4,100,0.562181,0.1919
25,2017-05-24,Options,USD,AAPL,30JUN17,P,144.0,-4,100,0.74778254,0.4273
30,2017-06-12,Options,USD,AAPL,21JUL17,P,125.0,3,100,0.467931,0.063
33,2017-06-12,Options,USD,AAPL,21JUL17,P,130.0,-3,100,0.83203069,0.1251
